## Predicting winners among undervalued public firms based on fundamental data

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, auc
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline

### Problem Statement

    Based on the publicly available financial accounting information, we want to identify winners and losers among undervalued (i.e., firms with higher book value than market value) public firms. We will use the fundamental dataset available on kaggle to calculate metrics based on a research done by UChicago on value investing. The fundamental dataset covers financial data on 400+ firms over a 4 year period between 2012 and 2015. Further, we will build a model to determine whether the metrics can predict winners and losers among undervalued firms. 

### Source

- Metrics from fundamental data is based on UChicago reasearch paper on value investing - "The Use of Historical Financial Statement Information
to Separate Winners from Losers - Joseph D. Piotroski" (https://www.chicagobooth.edu/~/media/FE874EE65F624AAEBD0166B1974FD74D.pdf)
- Fundamental data from kaggle (https://www.kaggle.com/dgawlik/nyse)

### Dataset Description

#### Fundamental data

Variable |Type of Variable |Description
:- |:- |:-
Id|numeric|line id
Ticker Symbol|text|publicly traded symbol name
Period Ending|text|financial year ending date
Accounts Payable|numeric|
Accounts Receivable|numeric|
Add'l income/expense items|numeric|
After Tax ROE|numeric|
Capital Expenditures|numeric|
Capital Surplus|numeric|
Cash Ratio|numeric|
Cash and Cash Equivalents|numeric|
Changes in Inventories|numeric|
Common Stocks|numeric|
Cost of Revenue|numeric|
Current Ratio|numeric|
Deferred Asset Charges|numeric|
Deferred Liability Charges|numeric|
Depreciation|numeric|
Earnings Before Interest and Tax|numeric|
Earnings Before Tax|numeric|
Effect of Exchange Rate|numeric|
Equity Earnings/Loss Unconsolidated Subsidiary|numeric|
Fixed Assets|numeric|
Goodwill|numeric|
Gross Margin|numeric|
Gross Profit|numeric|
Income Tax|numeric|
Intangible Assets|numeric|
Interest Expense|numeric|
Inventory|numeric|
Investments|numeric|
Liabilities|numeric|
Long-Term Debt|numeric|
Long-Term Investments|numeric|
Minority Interest|numeric|
Misc. Stocks|numeric|
Net Borrowings|numeric|
Net Cash Flow|numeric|
Net Cash Flow-Operating|numeric|
Net Cash Flows-Financing|numeric|
Net Cash Flows-Investing|numeric|
Net Income|numeric|
Net Income Adjustments|numeric|
Net Income Applicable to Common Shareholders|numeric|
Net Income-Cont. Operations|numeric|
Net Receivables|numeric|
Non-Recurring Items|numeric|
Operating Income|numeric|
Operating Margin|numeric|
Other Assets|numeric|
Other Current Assets|numeric|
Other Current Liabilities|numeric|
Other Equity|numeric|
Other Financing Activities|numeric|
Other Investing Activities|numeric|
Other Liabilities|numeric|
Other Operating Activities|numeric|
Other Operating Items|numeric|
Pre-Tax Margin|numeric|
Pre-Tax ROE|numeric|
Profit Margin|numeric|
Quick Ratio|numeric|accounting ratio
Research and Development|numeric|
Retained Earnings|numeric|
Sale and Purchase of Stock|numeric|
"Sales General and Admin."|numeric|
Short-Term Debt / Current Portion of Long-Term Debt|numeric|
Short-Term Investments|numeric|
Total Assets|numeric|
Total Current Assets|numeric|
Total Current Liabilities|numeric|
Total Equity|numeric|
Total Liabilities|numeric|
Total Liabilities & Equity|numeric|
Total Revenue|numeric|
Treasury Stock|numeric|
For Year|numeric|
Earnings Per Share|numeric|
Estimated Shares Outstanding|numeric|


#### Prices data

Variable|Type of Variable|Description
:- |:- |:-
date|text|trading date
symbol|text|symbol traded
open|numeric|opening price
close|numeric|closing price
low|numeric|low price of the day
high|numeric|high price of the day
volume|numeric|traded volume

#### Securities data

Variable|Type of Variable|Description
:-|:-|:-
"Ticker symbol"|text|traded symbol
"Security"|text|company name
"SEC filings"|text|filing type
"GICS Sector"|text|sector that the company belongs
"GICS Sub Industry"|text|sub industry category
"Address of Headquarters"|text|address
"Date first added"|text|issued date
"CIK"|text|central index key


### Risks and Assumptions

    - Data contains accurate information
    - Minimum 2 years of information is needed to calculate metrics
    - There exists inefficiency with market pricing of undervalued firms
    - Data may not correctly reflect corporate action information
    - Dataset may not be sufficient
    - Dataset for a particular sector/industry may not be sufficient

### Success Metrics

    - Able to clean, munge, combine different data sets, and identify undervalued firms
    - Classify winners/losers based on FSCORE metrics and show whether there is meaningful association or not

### Hypothesis

Null Hypothesis: 
       
    There is no association between future win/loss of undervalued firms and FSCORE metrics obtained from financial accounting data
    
Alternate Hypothesis:

    FSCORE metrics from financial accounting data can help predict future win/loss of undervalued firms

### Exploratory Analysis (WORK IN PROGRESS!!!!)

In [ ]:
plt.style.use("fivethirtyeight")
plt.rcParams["figure.figsize"] = [8,5]

In [29]:
df_fundamentals=pd.read_csv("../data/fundamentals.csv")
df_fundamentals.head()

,Unnamed: 0,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,...,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding
0,0,AAL,2012-12-31,3.068000e+09,-222000000.0,-1.961000e+09,23.0,-1.888000e+09,4.695000e+09,53.0,...,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,2012.0,-5.60,3.350000e+08
1,1,AAL,2013-12-31,4.975000e+09,-93000000.0,-2.723000e+09,67.0,-3.114000e+09,1.059200e+10,75.0,...,1.432300e+10,1.380600e+10,-2.731000e+09,4.500900e+10,4.227800e+10,2.674300e+10,0.0,2013.0,-11.25,1.630222e+08
2,2,AAL,2014-12-31,4.668000e+09,-160000000.0,-1.500000e+08,143.0,-5.311000e+09,1.513500e+10,60.0,...,1.175000e+10,1.340400e+10,2.021000e+09,4.120400e+10,4.322500e+10,4.265000e+10,0.0,2014.0,4.02,7.169154e+08
3,3,AAL,2015-12-31,5.102000e+09,352000000.0,-7.080000e+08,135.0,-6.151000e+09,1.159100e+10,51.0,...,9.985000e+09,1.360500e+10,5.635000e+09,4.278000e+10,4.841500e+10,4.099000e+10,0.0,2015.0,11.39,6.681299e+08
4,4,AAP,2012-12-29,2.409453e+09,-89482000.0,6.000000e+05,32.0,-2.711820e+08,5.202150e+08,23.0,...,3.184200e+09,2.559638e+09,1.210694e+09,3.403120e+09,4.613814e+09,6.205003e+09,-27095000.0,2012.0,5.29,7.328355e+07


In [30]:
df_fundamentals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1781 entries, 0 to 1780
Data columns (total 79 columns):
Unnamed: 0                                             1781 non-null int64
Ticker Symbol                                          1781 non-null object
Period Ending                                          1781 non-null object
Accounts Payable                                       1781 non-null float64
Accounts Receivable                                    1781 non-null float64
Add'l income/expense items                             1781 non-null float64
After Tax ROE                                          1781 non-null float64
Capital Expenditures                                   1781 non-null float64
Capital Surplus                                        1781 non-null float64
Cash Ratio                                             1482 non-null float64
Cash and Cash Equivalents                              1781 non-null float64
Changes in Inventories                            

##### Create date index using period ending

In [31]:
df_fundamentals.columns = [c.replace(' ', '_') for c in df_fundamentals.columns]
df_fundamentals['Period_Ending'] = pd.to_datetime(df_fundamentals['Period_Ending'])
df_fundamentals.set_index('Period_Ending', inplace=True)

In [32]:
df_fundamentals.head()

,Unnamed:_0,Ticker_Symbol,Accounts_Payable,Accounts_Receivable,Add'l_income/expense_items,After_Tax_ROE,Capital_Expenditures,Capital_Surplus,Cash_Ratio,Cash_and_Cash_Equivalents,...,Total_Current_Assets,Total_Current_Liabilities,Total_Equity,Total_Liabilities,Total_Liabilities_&_Equity,Total_Revenue,Treasury_Stock,For_Year,Earnings_Per_Share,Estimated_Shares_Outstanding
Period_Ending,,,,,,,,,,,,,,,,,,,,,
2012-12-31,0,AAL,3.068000e+09,-222000000.0,-1.961000e+09,23.0,-1.888000e+09,4.695000e+09,53.0,1.330000e+09,...,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,2012.0,-5.60,3.350000e+08
2013-12-31,1,AAL,4.975000e+09,-93000000.0,-2.723000e+09,67.0,-3.114000e+09,1.059200e+10,75.0,2.175000e+09,...,1.432300e+10,1.380600e+10,-2.731000e+09,4.500900e+10,4.227800e+10,2.674300e+10,0.0,2013.0,-11.25,1.630222e+08
2014-12-31,2,AAL,4.668000e+09,-160000000.0,-1.500000e+08,143.0,-5.311000e+09,1.513500e+10,60.0,1.768000e+09,...,1.175000e+10,1.340400e+10,2.021000e+09,4.120400e+10,4.322500e+10,4.265000e+10,0.0,2014.0,4.02,7.169154e+08
2015-12-31,3,AAL,5.102000e+09,352000000.0,-7.080000e+08,135.0,-6.151000e+09,1.159100e+10,51.0,1.085000e+09,...,9.985000e+09,1.360500e+10,5.635000e+09,4.278000e+10,4.841500e+10,4.099000e+10,0.0,2015.0,11.39,6.681299e+08
2012-12-29,4,AAP,2.409453e+09,-89482000.0,6.000000e+05,32.0,-2.711820e+08,5.202150e+08,23.0,5.981110e+08,...,3.184200e+09,2.559638e+09,1.210694e+09,3.403120e+09,4.613814e+09,6.205003e+09,-27095000.0,2012.0,5.29,7.328355e+07


In [37]:
df_fundamentals[df_fundamentals.Ticker_Symbol == 'AAL'][['Total_Assets','Total_Liabilities','Total_Equity']]

,Total_Assets,Total_Liabilities,Total_Equity
Period_Ending,,,
2012-12-31,2.351000e+10,2.489100e+10,-7.987000e+09
2013-12-31,4.227800e+10,4.500900e+10,-2.731000e+09
2014-12-31,4.322500e+10,4.120400e+10,2.021000e+09
2015-12-31,4.841500e+10,4.278000e+10,5.635000e+09


In [86]:
df_prices=pd.read_csv("../data/prices.csv")
df_prices['date'] = pd.to_datetime(df_prices['date'])
df_prices.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [87]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null datetime64[ns]
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 45.5+ MB
